In [ ]:
import csv
import json
import os

import cache
import dependents
import version

In [ ]:
github_cache = cache.GithubAPI(login_or_token=os.environ.get("GH_TOKEN"))

In [ ]:
config = {}
with open('config.json') as json_file:
    config = json.load(json_file)

In [ ]:
for index in range(len((config["repos"]))):
    print("Repo nr: ", index)
    repo = config["repos"][index]
    print(repo["name"])
    package_meta = dependents.per_package(repo, github_cache)
    print(
        "Cached/Uncached: ",
        github_cache.cached_calls,
        "/",
        github_cache.uncached_calls
    )
    print(
        "Total calls: ",
        github_cache.cached_calls + github_cache.uncached_calls
    )
    
    repo["package_meta"] = package_meta

# Get versions used

In [ ]:
jsly_language = ['Node.js', 'Typescript', 'Javascript', 'Wasm']

for repo in config["repos"]:
    for index, package in enumerate(repo['package_meta']):
        print(package['name'])
        print(repo['packages'][package['name']])
        
        language = repo['packages'][package['name']]
        func = version.get_rust_lib_version
        if language in jsly_language:
            func = version.get_js_lib_version

        for (name, metadata) in package['active_dependents']:
        #if (True):
        #    (name, metadata) = package['active_dependents'][8]
            print(name)
            metadata['versions'] = func(github_cache, name, package['name'])

In [ ]:
print(
        "Cached/Uncached: ",
        github_cache.cached_calls,
        "/",
        github_cache.uncached_calls
    )
print(
        "Total calls: ",
        github_cache.cached_calls + github_cache.uncached_calls
)

# Create version dict

In [ ]:
for repo in config["repos"]:
    for package in repo['package_meta']:
        locked_versions = {}
        for (name, metadata) in package['active_dependents']:
            print(name)
            print(len(metadata['versions']))
            for version in metadata['versions']:
                version = version['locked_version']
                print(version)
                if version in locked_versions:
                    locked_versions[version] += 1
                else:
                    locked_versions[version] = 1
        package['locked_versions_histo'] = locked_versions


In [ ]:
with open('metrics.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile)
        for repo in config['repos']:
                filewriter.writerow([repo['org'] + '/' + repo['name']])
                filewriter.writerow([])
                filewriter.writerow([
                        'Package',
                        'Language',
                        'Dependents',
                        'Active Dependents'
                ])
                for package in repo['package_meta']:
                        package_name = package['name']

                        filewriter.writerow([
                                package_name,
                                repo['packages'][package_name],
                                len(package['dependents']),
                                len(package['active_dependents'])
                        ])

                        filewriter.writerow([])

                        # Replace None with Unknown
                        keys = [
                                'Unknown' if x is None else x for x in package['locked_versions_histo'].keys()
                        ]

                        filewriter.writerow(keys)
                        filewriter.writerow(package['locked_versions_histo'].values())

                filewriter.writerow([])
                filewriter.writerow([])

# Export Config

In [ ]:
import copy

config_copy = copy.deepcopy(config)
for repo in config_copy["repos"]:
    for package in repo['package_meta']:
        for (name, metadata) in package['active_dependents']:
            if 'commit_activity' in metadata:
                del metadata['commit_activity']
        for key, value in package['filtered_dependents'].items():
            if 'commit_activity' in value:
                del value['commit_activity']

with open("store_versions.json", "w") as outfile:
    json.dump(config_copy, outfile, indent=4)

In [ ]:
config = {}
with open('store.json') as json_file:
    config = json.load(json_file)